In [ ]:
!conda create -n py38 python=3.8 -y

In [1]:
!conda env list

# conda environments:
#
base                     /opt/conda



In [8]:
!/opt/conda/envs/py38/bin/python3 -V

Python 3.8.19


In [ ]:
#!git clone https://github.com/duyndh98/PPFR.git
#!rm -rf TFace
#!git clone https://github.com/Tencent/TFace.git

In [ ]:
!/opt/conda/envs/py38/bin/pip install dareblopy

In [ ]:
!/opt/conda/envs/py38/bin/pip install torchjpeg

In [ ]:
!/opt/conda/envs/py38/bin/pip install pyyaml

In [ ]:
!/opt/conda/envs/py38/bin/pip install tensorboard

In [ ]:
!/opt/conda/envs/py38/bin/pip install opencv-python

In [ ]:
!/opt/conda/envs/py38/bin/pip install protobuf==3.20.*

In [24]:
import os

DATASET_PATH = '/kaggle/input/vietnamese-celebrity-faces'

INDEX_ROOT = '/kaggle/working/index_files'
if not os.path.exists(INDEX_ROOT):
    os.mkdir(INDEX_ROOT)
    
index_path = os.path.join(INDEX_ROOT, 'vietnamese-celebrity-faces.index.txt')
if os.path.exists(index_path):
    os.remove(index_path)

celeb_id = 0
with open(index_path, 'w') as out_file:
    for job_name in os.listdir(DATASET_PATH):
        job_path = os.path.join(DATASET_PATH, job_name)
        for celeb_name in os.listdir(job_path):
            celeb_path = os.path.join(job_path, celeb_name)
            for img_name in os.listdir(celeb_path):
                out_file.write("{}\t{}\n".format(os.path.join(job_name, celeb_name, img_name), celeb_id))
            celeb_id += 1

In [25]:
!cp -r '/kaggle/working/TFace/recognition' '/kaggle/working'

In [16]:
import os
os.chdir('/kaggle/working/recognition/tasks/minusface')
os.getcwd()

'/kaggle/working/recognition/tasks/minusface'

In [17]:
%%writefile '/kaggle/working/recognition/torchkit/hooks/learning_rate_hook.py'
import logging
from bisect import bisect
from .base_hook import Hook


def set_optimizer_lr(optimizer, lr):
    if isinstance(optimizer, dict):
        backbone_opt, head_opts = optimizer['backbone'], optimizer['heads']
        if len(backbone_opt.param_groups) > 1: # is stage 1
            rate = 0.1
            backbone_opt.param_groups[0]['lr'] = lr * rate
            backbone_opt.param_groups[1]['lr'] = lr
            for _, head_opt in head_opts.items():
                for param_group in head_opt.param_groups:
                    param_group['lr'] = lr
        else: # is stage 2
            for param_group in backbone_opt.param_groups:
                param_group['lr'] = lr
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


def warm_up_lr(step, warmup_step, init_lr, optimizer):
    """ Warm up learning rate when batch step below warmup steps
    """

    lr = step * init_lr / warmup_step
    if step % 500 == 0:
        logging.info("Current step {}, learning rate {}".format(step, lr))

    set_optimizer_lr(optimizer, lr)


def adjust_lr(epoch, learning_rates, stages, optimizer):
    """ Decay the learning rate based on schedule
    """

    pos = bisect(stages, epoch)
    lr = learning_rates[pos]
    logging.info("Current epoch {}, learning rate {}".format(epoch + 1, lr))

    set_optimizer_lr(optimizer, lr)


class LearningRateHook(Hook):
    """ LearningRate Hook, adjust learning rate in training
    """
    def __init__(self,
                 learning_rates,
                 stages,
                 warmup_step):
        """ Create a ``LearningRateHook`` object

            Args:
            learning_rates: all learning rates value
            stages: learning rate adjust stages value
            warmup_step: step num of warmup
        """

        self.learning_rates = learning_rates
        self.stages = stages
        if len(self.learning_rates) != len(self.stages) + 1:
            raise RuntimeError("Learning_rates size should be one larger than stages size")
        self.init_lr = self.learning_rates[0]
        self.warmup_step = warmup_step

    def before_train_iter(self, task, step, epoch):
        global_step = epoch * task.step_per_epoch + step
        if self.warmup_step > 0 and global_step <= self.warmup_step:
            warm_up_lr(global_step, self.warmup_step, self.init_lr, task.opt)

    def before_train_epoch(self, task, epoch):
        adjust_lr(epoch, self.learning_rates, self.stages, task.opt)


Overwriting /kaggle/working/recognition/torchkit/hooks/learning_rate_hook.py


In [24]:
%%writefile '/kaggle/working/recognition/tasks/minusface/train.yaml'
SEED: 1337 # random seed for reproduce results
DATA_ROOT: '/kaggle/input/vietnamese-celebrity-faces' # [fill in this blank] the parent directory where your train/val/test data are stored
INDEX_ROOT: '/kaggle/working/index_files' # [fill in this blank] the parent directory for index
DATASETS:
  - name: 'vietnamese-celebrity-faces.index'# [fill in this blank] the name of your dataset
    batch_size: 16
    weight: 1.0
    scale: 64
    margin: 0.5

BACKBONE_RESUME: ""
HEAD_RESUME: ""
META_RESUME: ""

INPUT_SIZE: [ 112, 112 ]
BACKBONE_NAME: 'IR_18' # support: ['IR_18', 'IR_50']
EMBEDDING_SIZE: 512

MODEL_ROOT: '/kaggle/working/ckpt_s1' # the root to buffer your checkpoints
LOG_ROOT: '/kaggle/working/tensorboard_s1' # the root to log your train/val status

DIST_FC: true
HEAD_NAME: "ArcFace" # support:  ['ArcFace', 'CurricularFace', 'CosFace']
LOSS_NAME: 'DistCrossEntropy' # support: ['DistCrossEntropy', 'Softmax']

RGB_MEAN: [ 0.5, 0.5, 0.5 ] # for normalize inputs to [-1, 1]
RGB_STD: [ 0.5, 0.5, 0.5 ]

LRS: [ 0.01, 0.001, 0.0001, 0.00001 ]
WARMUP_STEP: -1
STAGES: [ 10, 18, 22 ]

START_EPOCH: 0 # start epoch
NUM_EPOCH: 18 # total epoch number
SAVE_EPOCHS: [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18 ]

WEIGHT_DECAY: 0.0005 # do not apply to batch_norm parameters
MOMENTUM: 0.9

WORLD_SIZE: 1
RANK: 0
LOCAL_RANK: 0
DIST_BACKEND: 'nccl'
DIST_URL: 'env://'

NUM_WORKERS: 8

AMP: false # fp16 for backbone

# MinusFace
METHOD: MinusFace
TASK: stage1 # toy, stage1, stage2
NUM_DUPS: 1
NUM_AUG: 3 # multiplier for data augmentation
TASK_BACKBONE: 'IR_18' # IR_18, IR_50
PRETRAIN_CKPT: '' # [fill in this blank] to train the recognition model requires pretrained MinusFace checkpoint
TASK_VER: 3

Overwriting /kaggle/working/recognition/tasks/minusface/train.yaml


In [25]:
!rm -rf '/kaggle/working/ckpt_s1'
!mkdir '/kaggle/working/ckpt_s1'
!rm -rf '/kaggle/working/tensorboard_s1'
!mkdir '/kaggle/working/tensorboard_s1'

In [44]:
%%writefile '/kaggle/working/recognition/tasks/partialface/base_task.py'
import os
import sys
import copy
import logging
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from torchkit.task.base_task import BaseTask
from torchkit.backbone import get_model
from torchkit.head import get_head
from torchkit.util import get_class_split, separate_resnet_bn_paras
from torchkit.data import MultiDataset, MultiDistributedSampler

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s: %(message)s')


class AugmentedMultiDataset(MultiDataset):
    def __init__(self, data_root, index_root, names, transform, num_aug, **kwargs) -> None:
        super().__init__(data_root, index_root, names, transform, **kwargs)
        self.num_aug = num_aug

    def _build_inputs(self, world_size=None, rank=None):
        """ Read index file and saved in ``self.inputs``
            If ``self.is_shard`` is True, ``total_sample_nums`` > ``sample_nums``
        """

        for i, name in enumerate(self.names):
            index_file = os.path.join(self.index_root, name + ".txt")
            self.index_parser.reset()
            self.inputs[name] = []
            with open(index_file, 'r') as f:
                for line_i, line in enumerate(f):
                    sample = self.index_parser(line)
                    if self.is_shard is False:
                        # ============== PartialFace / MinusFace ==============
                        for i in range(self.num_aug):
                            self.inputs[name].append(sample)
                        # =====================================================
                    else:
                        if line_i % world_size == rank:
                            self.inputs[name].append(sample)
                        else:
                            pass
            self.class_nums[name] = self.index_parser.class_num + 1
            self.sample_nums[name] = len(self.inputs[name])
            if self.is_shard:
                self.total_sample_nums[name] = self.index_parser.sample_num
                logging.info("Dataset %s, class_num %d, total_sample_num %d, sample_num %d" % (
                    name, self.class_nums[name], self.total_sample_nums[name], self.sample_nums[name]))
            else:
                logging.info("Dataset %s, class_num %d, sample_num %d" % (
                    name, self.class_nums[name], self.sample_nums[name]))


class LocalBaseTask(BaseTask):
    def __init__(self, cfg_file):
        super().__init__(cfg_file=cfg_file)
        # if self.cfg['METHOD'] == 'PartialFace':
        #     self.num_aug = self.cfg['NUM_AUG']
        #     self.num_chs = self.cfg['NUM_CHS']

    def make_inputs(self):
        """ make datasets
        """
        rgb_mean = self.cfg['RGB_MEAN']
        rgb_std = self.cfg['RGB_STD']
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((112, 112)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=rgb_mean, std=rgb_std)
        ])

        ds_names = list(self.branches.keys())
        # ============== PartialFace / MinusFace ==============
        ds = AugmentedMultiDataset(self.cfg['DATA_ROOT'], self.cfg['INDEX_ROOT'], ds_names,
                                   transform, self.cfg['NUM_AUG'])
        # =====================================================

        ds.make_dataset(shard=False)
        self.class_nums = ds.class_nums

        sampler = MultiDistributedSampler(ds, self.batch_sizes)
        self.train_loader = DataLoader(ds, sum(self.batch_sizes), shuffle=False,
                                       num_workers=self.cfg["NUM_WORKERS"], pin_memory=True,
                                       sampler=sampler, drop_last=False)

        self.step_per_epoch = len(self.train_loader)
        logging.info("Step_per_epoch = %d" % self.step_per_epoch)

    def make_model(self):
        """ build training backbone and heads
        """

        # ============== PartialFace / MinusFace ==============
        if self.cfg['METHOD'] == 'PartialFace':

            backbone_name = self.cfg['BACKBONE_NAME']
            backbone_model = get_model(backbone_name)
            self.backbone = backbone_model(self.input_size)

            self.backbone.input_layer = nn.Sequential(nn.Conv2d(self.cfg['NUM_CHS'] * 3, 64, (3, 3), 1, 1, bias=False),
                                                      nn.BatchNorm2d(64), nn.PReLU(64))

            logging.info("{} Backbone Generated".format(backbone_name))

        else:  # self.cfg['METHOD'] == 'MinusFace'
            from tasks.minusface.minusface import MinusBackbone

            generator, recognizer = None, None

            recognizer = get_model(self.cfg['TASK_BACKBONE'])([112, 112])
            print('Recognizer is {}'.format(self.cfg['TASK_BACKBONE']))

            if self.cfg['TASK'] == 'stage2':
                pretrain_backbone = MinusBackbone(mode='stage1',
                                                  recognizer=get_model(self.cfg['TASK_BACKBONE'])([112, 112]))
                pretrain_backbone.load_state_dict(torch.load(self.cfg['PRETRAIN_CKPT']))
                pretrain_backbone.generator.mode = self.cfg['TASK']
                generator = copy.deepcopy(pretrain_backbone.generator)
                print('Load pretrain ckpt: ', self.cfg['PRETRAIN_CKPT'])

            self.backbone = MinusBackbone(mode=self.cfg['TASK'], n_duplicate=1, generator=generator,
                                          recognizer=recognizer)
            logging.info(f"Minus {self.cfg['TASK']} Backbone Generated")
        # =====================================================

        self.backbone.cuda()

        embedding_size = self.cfg['EMBEDDING_SIZE']
        self.class_shards = []
        metric = get_head(self.cfg['HEAD_NAME'], dist_fc=self.dist_fc)

        for name, branch in self.branches.items():
            class_num = self.class_nums[name]
            class_shard = get_class_split(class_num, self.world_size)
            self.class_shards.append(class_shard)
            logging.info('Split FC: {}'.format(class_shard))

            init_value = torch.FloatTensor(embedding_size, class_num)
            init.normal_(init_value, std=0.01)
            head = metric(in_features=embedding_size,
                          gpu_index=self.rank,
                          weight_init=init_value,
                          class_split=class_shard,
                          scale=branch.scale,
                          margin=branch.margin)
            del init_value
            head = head.cuda()
            self.heads[name] = head

    def get_optimizer(self):
        """ build optimizers
        """

        learning_rates = self.cfg['LRS']
        init_lr = learning_rates[0]
        weight_decay = self.cfg['WEIGHT_DECAY']
        momentum = self.cfg['MOMENTUM']

        # ===================== MinusFace =====================
        if self.cfg['METHOD'] == 'Minusface':
            if self.cfg['TASK'] == 'stage2':
                backbone_opt = optim.SGD([{'params': self.backbone.recognizer.parameters(), 'lr': init_lr}],
                                         weight_decay=weight_decay, lr=init_lr, momentum=momentum)
            else:
                backbone_opt = optim.SGD([{'params': self.backbone.generator.parameters(), 'lr': init_lr / 10.},
                                          {'params': self.backbone.recognizer.parameters(), 'lr': init_lr}],
                                         weight_decay=weight_decay, lr=init_lr, momentum=momentum)
        else:
            backbone_paras_only_bn, backbone_paras_wo_bn = separate_resnet_bn_paras(self.backbone)
            backbone_opt = optim.SGD([
                {'params': backbone_paras_wo_bn, 'weight_decay': weight_decay},
                {'params': backbone_paras_only_bn}], lr=init_lr, momentum=momentum)
        # =====================================================

        head_opts = OrderedDict()
        for name, head in self.heads.items():
            opt = optim.SGD([{'params': head.parameters()}], lr=init_lr, momentum=momentum,
                            weight_decay=weight_decay)
            head_opts[name] = opt

        optimizer = {
            'backbone': backbone_opt,
            'heads': head_opts,
        }
        return optimizer

    def loop_step(self, epoch):
        """ Implemented by sub class, which run in every training step
        """
        raise NotImplementedError()

    def train(self):
        raise NotImplementedError()


Overwriting /kaggle/working/recognition/tasks/partialface/base_task.py


In [26]:
!export CUDA_VISIBLE_DEVICES='0'
!/opt/conda/envs/py38/bin/python3 -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 /kaggle/working/recognition/tasks/minusface/train.py

/opt/conda/envs/py38/lib/python3.8/site-packages/torch/distributed/launch.py:181: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
2024-08-13 00:41:42,060: Dataset vietnamese-celebrity-faces.index, batch_size 16, weight 1.000000, scale 64, margin 0.500000
2024-08-13 00:41:42,106: Added key: store_based_barrier_key:1 to store for rank: 0
2024-08-13 00:41:42,106: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
2024-08-13 00:41:42,131: world_size: 1, rank: 0, local_rank: 0
SEED 1337
DATA_ROOT /kaggle/input/vietnamese-celebrity-faces
INDEX_ROOT /kaggle/working/index_files
DATASETS [{'name': 'vietnamese-celebri